In [2]:
from openai import OpenAI

In [5]:
client = OpenAI(api_key="lm-studio", base_url="http://localhost:1234/v1")

response = client.chat.completions.create(
    model="llama-3.2-3b-instruct",
    messages=[{"role": "user", "content": "Hello!"}],
)

print(response.choices[0].message.content)

How can I assist you today?
